# Example creating Monitoring Location

This example demonstrates how to create a monitoring location XML file. The resulting file can be submitted using `wqxweblib` or the WQX Web application on CDX.

Start by importing the elements which are needed in your Monitoring Location.

In [1]:
!pip3 install --upgrade wqxlib

from wqxlib import (
    Document,
    Header,
    Payload,
    Submission
)
from wqxlib.wqx_v3_0 import (
    MonitoringLocation,
    MonitoringLocationGeospatial,
    MonitoringLocationIdentity,
    Organization,
    OrganizationDescription,
    WQX
)


Defaulting to user installation because normal site-packages is not writeable
  Using cached wqxlib-3.0.0.3.1-py3-none-any.whl (92 kB)
  Using cached yattag-1.14.0-py3-none-any.whl
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached rfc3986-1.4.0-py2.py3-none-any.whl (31 kB)


There are many ways to generate the document, but basically you just need to specify its properties and run `generateXML()` on it. In this example, we'll create objects representing distinct portions of the submission, then combine them together at the end before generating the XML.

Let's start by defining the organization which is in charge of the Monitoring Location.

In [2]:
organization_description = OrganizationDescription(
    organizationIdentifier = "WQXTEST",
    organizationFormalName = "WQX Test Organization",
    organizationDescriptionText = "Test"
)

The Geospatial information about the Monitoring Location is similarly defined.

In [3]:
monitoring_location_geospatial = MonitoringLocationGeospatial(
    latitudeMeasure = "38.6470",
    longitudeMeasure = "-82.8587",
    sourceMapScale = "2400",
    horizontalCollectionMethodName = "Interpolation-Map",
    horizontalCoordinateReferenceSystemDatumName = "NAD83",
    countryCode = "US",
    stateCode = "WV",
    countyCode = "039"
)

We also need to describe the Monitoring Location's identity.

In [4]:
monitoring_location_identity = MonitoringLocationIdentity(
    monitoringLocationIdentifier = "GREENUP",
    monitoringLocationName = "Greenup Dam",
    monitoringLocationTypeName = "River/Stream",
    hucEightDigitCode = "05090103",
    hucTwelveDigitCode = "050901030107",
    tribalLandIndicator = False
)

The last bit of information we need to add is header metadata which lets you tell WQX who to email about the status of the submission.

In [5]:
header = Header(
    author = "Test Author",
    organization = "Test Organization",
    contactInfo = "Test Organization Mailing or Physical Address",
    notification = "test@example.org"
)

Now we'll piece together these objects into a `Submission` object.

In [6]:
submission = Submission(
    document=Document(
        id='20201209ML8',
        header=header,
        # Even though we are assigning a single object to the payload key here,
        # it becomes a list item which can be appended to later
        payload=Payload(
            operation='Update-Insert',
            wqx=WQX(
                organization=Organization(
                    organizationDescription = organization_description,
                    monitoringLocation=MonitoringLocation(
                        monitoringLocationGeospatial=monitoring_location_geospatial,
                        monitoringLocationIdentity=monitoring_location_identity
                    )
                )
            )
        )
    )
)

A few things to note:

- The `id` attribute should be an identifier of your choosing which is unique within the organization.
- The `operation` attribute should almost always be `Update-Insert` unless you are deleting something.
- The `payload` attribute can be either a list or a single `Payload` object. If you assign a single object, it will be converted into a list of `Payload` objects containing only the one object. This convention allows you to later add additional `Payload` objects, if that is appropriate for your project. You can enclose the `Payload` object with square brackets to remind yourself of what is happening if you choose.

Now that the required attributes are already assigned, we can generate the XML file.

In [7]:
doc = submission.document.generateXML()
print(doc)

<?xml version="1.0" encoding="UTF-8"?>
<Document Id="20201209ML8" xmlns="http://www.exchangenetwork.net/schema/v1.0/ExchangeNetworkDocument.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <Header>
    <Author>Test Author</Author>
    <Organization>Test Organization</Organization>
    <Title>WQX</Title>
    <CreationTime>2021-03-10T19:44:53+00:00</CreationTime>
    <ContactInfo>Test Organization Mailing or Physical Address</ContactInfo>
    <Notification>test@example.org</Notification>
  </Header>
  <Payload Operation="Update-Insert">
    <WQX xmlns="http://www.exchangenetwork.net/schema/wqx/3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.exchangenetwork.net/schema/wqx/3 http://www.exchangenetwork.net/schema/wqx/3/index.xsd">
      <Organization>
        <OrganizationDescription>
          <OrganizationIdentifier>WQXTEST</OrganizationIdentifier>
          <OrganizationFormalName>WQX Test Organization</OrganizationFormalName>
        

That's it! You could write that document to a file and submit it directly to WQX Web to import into WQX. If you want to automate that process, take a look at `wqxweblib` which simplifies the process of submitting data to WQX Web's API.

You may notice that the XML document closely reflects the structure of the object you created. A future version of the library will abstract this process a bit more so you can focus on the data submission and less on the format of the document.